In [1]:
import numpy as np
np.random.seed(0)

#Activation functions

def sin_function(x):
    return np.sin(x)

def sin_function_derivative(x):
    return np.cos(x)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x <= 0, 0, 1)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x)**2

#Function to approximate
def f(x):
    return np.sin(3*np.pi*x + 3*np.pi/20) * np.cos(2*np.pi*x + np.pi/10) + 2

In [2]:
#Function to build the mass matrix - M
from scipy.integrate import quad

def mass_matrix(func,sigma, weights, biases, n):
    M = np.zeros([n, n])
    
    for i in range(0, n):
        for j in range(0, n):
            M[i, j] = quad(func, 0, 1, args=(weights, biases,i+1,j+1,sigma), limit=1000)[0]

    return M

In [3]:
#Single basis function - Neural Network
def single_neural_net(x,weights,biases,i, sigma): 

    return sigma(weights[i]*x+biases[i])


In [4]:
#Basis function multiplication helper for quadrature
def double_neural_net(x,weights,biases,i,j, sigma): 

    return single_neural_net(x,weights,biases,i,sigma)*single_neural_net(x,weights,biases,j,sigma)

In [5]:
#Bias helper function for quadrature
def b_neural_net(x, weights, biases,i, sigma,f):
    return single_neural_net(x,weights,biases,i, sigma)*f(x)


In [6]:
#quadrature for the bias matrix - B
def b_matrix(func,sigma, weights, biases,f, neurons):
    b = np.zeros([neurons, 1])
    for i in range(0, neurons):
            b[i] = quad(func, 0, 1, args=(weights, biases,i,sigma,f), limit=1000)[0]

    return b

In [7]:
#Function to build the approximation vector
def u_nn(x, weights, biases, sigma,collocation, neurons, a):
    u = np.zeros(collocation)
    
    for j in range(0,collocation):
        for i in range(0,neurons):
            
            u[j] += single_neural_net(x[j], weights, biases, i, sigma)*a[i]
        
    return u


In [8]:
#helper function for eigenvalues and eigenvectors
from scipy.linalg import eig
def get_eig(M):
    D,V = eig(M, left=False, right=True)


    idx = D.argsort()[::-1]   
    D = D[idx]
    V = V[:,idx]

    D = np.flip(D)
    V = np.fliplr(V)

    return D,V

In [9]:
#Function to calculate L2 loss
def l2_loss(f_x, u_nn_result):
    return np.linalg.norm(f_x - u_nn_result)**2

In [10]:
#Function to build the Phi matrix, made up of n basis functions
def phi_matrix(x,weights,biases,sigma,neurons,collocation):
    phi = np.zeros((collocation,neurons))
    for i in range(0,neurons):
        for j in range(0,collocation):

            phi[j,i] = sigma(weights[i]*x[j]+biases[i])
    return phi


In [11]:
def evaluate_func_approx(x,f, weights, biases, sigma, neurons, collocation,verbose=False):
    phi = phi_matrix(x,weights,biases,sigma,neurons,collocation)
    f_func = f(x)
    a = np.linalg.pinv(phi)@f_func

    u = u_nn(x, weights, biases, sigma, collocation,neurons, a)

    loss = l2_loss(f_func, u)
    if verbose:
        print("="*20)
        print("Sigma:", sigma.__name__)
        print("Number of collocation points:", collocation)
        print("Number of neurons:", neurons)
        print("L2 Loss:", loss)
        print("="*20)
    return loss

In [12]:
import matplotlib.pyplot as plt

def compute_loss_values(neurons, collocations, sigmas, Rs,xmin=0,xmax=1,verbose=False):
    loss_values = []
    #np.random.seed(0)  # Set a seed for reproducibility
    for sigma in sigmas:
        for neuron in neurons:
            for collocation in collocations:
                for R in Rs:
                    x = np.linspace(xmin, xmax, collocation)
                    weights = np.random.uniform(-R, R, neuron)
                    biases = np.random.uniform(-R, R, neuron)
                    loss = evaluate_func_approx(x, f, weights, biases, sigma, neuron, collocation, verbose=verbose)
                    loss_values.append((loss, collocation, neuron, sigma.__name__,R))  # Add activation function name
    return loss_values

In [13]:
import inspect
def get_var_name(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var][0]

In [47]:
import pandas as pd

def export_list(list,list_name,columns= ["Loss","# of Collocation","# of Neurons","Activation Function","R - Sample Range"]):
    file_name = f"{list_name}.xlsx"
    df = pd.DataFrame(list, columns=columns)
    df.to_csv(file_name, index=False)
    print(f"Data exported to {file_name}")
    

In [14]:
def print_loss_by_collocation(loss_values):
    """
    Prints the loss values grouped by collocation point, activation function, and range R.
    
    Parameters:
    loss_values (list of tuples): Each tuple contains (loss, collocation, neuron, activation_func_name, R).
    """
    # Initialize the dictionary to store data for each collocation point
    collocation_loss_data = {collocation: [] for _, collocation, _, _, _ in loss_values}

    # Populate the dictionary with (neuron, loss, activation_func_name, R) tuples for each collocation point
    for loss, collocation, neuron, activation_func_name, R in loss_values:
        collocation_loss_data[collocation].append((neuron, loss, activation_func_name, R))

    # Iterate over the unique activation function names
    for activation_func_name in set(activation_func_name for _, _, _, activation_func_name, _ in loss_values):
        print(f"Activation Function: {activation_func_name}")
        
        # Iterate over the collocation points and print the relevant data
        for collocation, data in collocation_loss_data.items():
            print(f"# of Collocation points: {collocation}")
            
            # Print the neurons, loss values, and R for the current activation function
            for neuron, loss, act_func_name, R in data:
                if act_func_name == activation_func_name:
                    print(f"  # of Neurons: {neuron}, Loss: {loss:.10e}, R: {R}")



In [15]:
def print_loss_by_neuron(loss_values):
    """
    Prints the loss values grouped by neuron, activation function, and range R.
    
    Parameters:
    loss_values (list of tuples): Each tuple contains (loss, collocation, neuron, activation_func_name, R).
    """
    # Initialize the dictionary to store data for each neuron
    neuron_loss_data = {neuron: [] for _, _, neuron, _, _ in loss_values}

    # Populate the dictionary with (collocation, loss, activation_func_name, R) tuples for each neuron
    for loss, collocation, neuron, activation_func_name, R in loss_values:
        neuron_loss_data[neuron].append((collocation, loss, activation_func_name, R))

    # Iterate over the unique activation function names
    for activation_func_name in set(activation_func_name for _, _, _, activation_func_name, _ in loss_values):
        print(f"Activation Function: {activation_func_name}")
        
        # Iterate over the neurons and print the relevant data
        for neuron, data in neuron_loss_data.items():
            print(f"# of Neurons: {neuron}")
            
            # Print the collocation points, loss values, and R for the current activation function
            for collocation, loss, act_func_name, R in data:
                if act_func_name == activation_func_name:
                    print(f"  # of Collocation points: {collocation}, Loss: {loss:.10e}, R: {R}")




In [16]:
def print_loss_by_R(loss_values):
    """
    Prints the loss values grouped by the range R, activation function, collocation points, and neurons.
    
    Parameters:
    loss_values (list of tuples): Each tuple contains (loss, collocation, neuron, activation_func_name, R).
    """
    # Initialize the dictionary to store data for each R value
    R_loss_data = {R: [] for _, _, _, _, R in loss_values}

    # Populate the dictionary with (collocation, neuron, loss, activation_func_name) tuples for each R value
    for loss, collocation, neuron, activation_func_name, R in loss_values:
        R_loss_data[R].append((collocation, neuron, loss, activation_func_name))

    # Iterate over the unique activation function names
    for activation_func_name in set(activation_func_name for _, _, _, activation_func_name, _ in loss_values):
        print(f"Activation Function: {activation_func_name}")
        
        # Iterate over the R values and print the relevant data
        for R, data in R_loss_data.items():
            print(f"Range R: {R}")
            
            # Print the collocation points, neurons, and loss values for the current activation function
            for collocation, neuron, loss, act_func_name in data:
                if act_func_name == activation_func_name:
                    print(f"  # of Collocation points: {collocation}, # of Neurons: {neuron}, Loss: {loss:.10e}")

In [17]:
collocation = 100
neurons = 50
sigma = sin_function
x = np.linspace(0,1,collocation)
weights = np.random.uniform(-1,1,neurons)
biases = np.random.uniform(-1,1,neurons)
test = evaluate_func_approx(x,f, weights, biases, sigma, neurons, collocation,verbose=True)

Sigma: sin_function
Number of collocation points: 100
Number of neurons: 50
L2 Loss: 0.08512083411934636


# Test 1 - Increasing #of neurons
Keeping collocations fixed at 50, and using ReLU, tanh, and sin as activation functions

In [48]:
neurons = [5,10,20,30,40,50,100]
sigmas = [relu_derivative,tanh_derivative,sin_function_derivative]
collocations = [50]
Rs = [1]
loss_values_increasing_neurons = compute_loss_values(neurons, collocations,sigmas,Rs,verbose=False)
print_loss_by_R(loss_values_increasing_neurons)
#loss_values

export_list(loss_values_increasing_neurons,"loss_values_increasing_neurons")

Activation Function: relu_derivative
Range R: 1
  # of Collocation points: 50, # of Neurons: 5, Loss: 6.2505626656e+00
  # of Collocation points: 50, # of Neurons: 10, Loss: 5.4622424458e+00
  # of Collocation points: 50, # of Neurons: 20, Loss: 6.1745839338e+00
  # of Collocation points: 50, # of Neurons: 30, Loss: 3.5907002681e+00
  # of Collocation points: 50, # of Neurons: 40, Loss: 3.5916586565e+00
  # of Collocation points: 50, # of Neurons: 50, Loss: 3.5743534093e+00
  # of Collocation points: 50, # of Neurons: 100, Loss: 1.4128812006e+00
Activation Function: sin_function_derivative
Range R: 1
  # of Collocation points: 50, # of Neurons: 5, Loss: 5.2540204831e+00
  # of Collocation points: 50, # of Neurons: 10, Loss: 1.9421282965e-01
  # of Collocation points: 50, # of Neurons: 20, Loss: 4.3044945804e-02
  # of Collocation points: 50, # of Neurons: 30, Loss: 4.4020868158e-02
  # of Collocation points: 50, # of Neurons: 40, Loss: 4.1909922053e-02
  # of Collocation points: 50, # 

# Test 2 - Increasing Collocation points
Keeping neurons fixed at 20, and the same activation functions

In [19]:
neurons = [20]
sigmas = [tanh,relu,sin_function]
collocations = [10,50,100,200,500]
Rs = [1]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,verbose=False)

In [20]:
print_loss_by_neuron(loss_values)

Activation Function: relu
# of Neurons: 20
  # of Collocation points: 10, Loss: 7.6679743793e-01, R: 1
  # of Collocation points: 50, Loss: 2.7933344949e+00, R: 1
  # of Collocation points: 100, Loss: 4.6528271642e+00, R: 1
  # of Collocation points: 200, Loss: 2.1875022779e+01, R: 1
  # of Collocation points: 500, Loss: 3.6592397331e+01, R: 1
Activation Function: tanh
# of Neurons: 20
  # of Collocation points: 10, Loss: 7.5071934566e-14, R: 1
  # of Collocation points: 50, Loss: 4.5618877679e-03, R: 1
  # of Collocation points: 100, Loss: 3.7128314277e-03, R: 1
  # of Collocation points: 200, Loss: 4.5068852383e-05, R: 1
  # of Collocation points: 500, Loss: 3.9335319413e-05, R: 1
Activation Function: sin_function
# of Neurons: 20
  # of Collocation points: 10, Loss: 1.9517762292e-05, R: 1
  # of Collocation points: 50, Loss: 4.7273167519e-02, R: 1
  # of Collocation points: 100, Loss: 8.1080991289e-02, R: 1
  # of Collocation points: 200, Loss: 1.5959541909e-01, R: 1
  # of Collocat

# Test 3 - Increasing the sampling range R
Keeping neurons and collocations fixed, increasing R to increase chance that w_i and b_i are further away from w_j and b_j

In [21]:
neurons = [30]
sigmas = [tanh,relu,sin_function]
sigmas_derivative = [tanh_derivative,relu_derivative,sin_function_derivative]
collocations = [50]
Rs = [1,2,4,5,10]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,verbose=False)
derivative_loss_values = compute_loss_values(neurons, collocations,sigmas_derivative,Rs,verbose=False)

In [22]:
derivative_loss_values

[(3.2283148588909507e-06, 50, 30, 'tanh_derivative', 1),
 (0.0006388915575742714, 50, 30, 'tanh_derivative', 2),
 (1.491234810888612e-06, 50, 30, 'tanh_derivative', 4),
 (3.8311405099109073e-08, 50, 30, 'tanh_derivative', 5),
 (4.0555510274317083e-10, 50, 30, 'tanh_derivative', 10),
 (2.57785105051447, 50, 30, 'relu_derivative', 1),
 (3.63050588629014, 50, 30, 'relu_derivative', 2),
 (2.759521964803343, 50, 30, 'relu_derivative', 4),
 (3.769551114827367, 50, 30, 'relu_derivative', 5),
 (1.7786649807567088, 50, 30, 'relu_derivative', 10),
 (0.04309152254064139, 50, 30, 'sin_function_derivative', 1),
 (0.0008959539660297248, 50, 30, 'sin_function_derivative', 2),
 (1.946806745467099e-05, 50, 30, 'sin_function_derivative', 4),
 (0.0004994071518351346, 50, 30, 'sin_function_derivative', 5),
 (0.0013129232026776543, 50, 30, 'sin_function_derivative', 10)]

In [23]:
print_loss_by_collocation(derivative_loss_values)

Activation Function: relu_derivative
# of Collocation points: 50
  # of Neurons: 30, Loss: 2.5778510505e+00, R: 1
  # of Neurons: 30, Loss: 3.6305058863e+00, R: 2
  # of Neurons: 30, Loss: 2.7595219648e+00, R: 4
  # of Neurons: 30, Loss: 3.7695511148e+00, R: 5
  # of Neurons: 30, Loss: 1.7786649808e+00, R: 10
Activation Function: sin_function_derivative
# of Collocation points: 50
  # of Neurons: 30, Loss: 4.3091522541e-02, R: 1
  # of Neurons: 30, Loss: 8.9595396603e-04, R: 2
  # of Neurons: 30, Loss: 1.9468067455e-05, R: 4
  # of Neurons: 30, Loss: 4.9940715184e-04, R: 5
  # of Neurons: 30, Loss: 1.3129232027e-03, R: 10
Activation Function: tanh_derivative
# of Collocation points: 50
  # of Neurons: 30, Loss: 3.2283148589e-06, R: 1
  # of Neurons: 30, Loss: 6.3889155757e-04, R: 2
  # of Neurons: 30, Loss: 1.4912348109e-06, R: 4
  # of Neurons: 30, Loss: 3.8311405099e-08, R: 5
  # of Neurons: 30, Loss: 4.0555510274e-10, R: 10


In [24]:
function_dict = {(func, r): err for err, _, _, func, r in loss_values}
derivative_dict = {(func, r): err for err, _, _, func, r in derivative_loss_values}

def base_function_name(derivative_name):
    if derivative_name.endswith('_derivative'):
        return derivative_name.replace('_derivative', '')
    return None

comparison_results = []
for (func_name, r), err in derivative_dict.items():
    base_name = base_function_name(func_name)
    if base_name:
        base_err = function_dict.get((base_name, r))
        if base_err is not None:
            comparison_results.append((base_name, r, base_err, err))

print(f"{'Function':<20}{'R Value':<10}{'Function Error':<20}{'Derivative Error':<25}")
print("="*80)
for base_name, r, base_err, der_err in comparison_results:
    print(f"{base_name:<20}{r:<10}{base_err:<20.10e}{der_err:<25.10e}")


Function            R Value   Function Error      Derivative Error         
tanh                1         2.3434375050e-05    3.2283148589e-06         
tanh                2         3.5857798345e-05    6.3889155757e-04         
tanh                4         2.3862848033e-09    1.4912348109e-06         
tanh                5         7.0521932236e-04    3.8311405099e-08         
tanh                10        1.1318563900e-03    4.0555510274e-10         
relu                1         1.1869997774e+00    2.5778510505e+00         
relu                2         2.8164332146e+00    3.6305058863e+00         
relu                4         4.5088958398e+00    2.7595219648e+00         
relu                5         2.9962341889e+00    3.7695511148e+00         
relu                10        4.0318911069e-01    1.7786649808e+00         
sin_function        1         4.5290035084e-02    4.3091522541e-02         
sin_function        2         8.5666752506e-04    8.9595396603e-04         
sin_function

In [25]:
neurons = [10,20,30,50,100]
sigmas = [tanh,relu,sin_function]
collocations = [5,10,30,50,100]
Rs = [1,2,4,5,10]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,verbose=False)

In [26]:
print_loss_by_collocation(loss_values)

Activation Function: relu
# of Collocation points: 5
  # of Neurons: 10, Loss: 3.7319916643e-01, R: 1
  # of Neurons: 10, Loss: 1.7828256458e-28, R: 2
  # of Neurons: 10, Loss: 7.8874320223e-01, R: 4
  # of Neurons: 10, Loss: 3.7319916643e-01, R: 5
  # of Neurons: 10, Loss: 8.6764838813e-27, R: 10
  # of Neurons: 20, Loss: 9.5649384758e-29, R: 1
  # of Neurons: 20, Loss: 1.3459939195e-29, R: 2
  # of Neurons: 20, Loss: 1.8143800820e-28, R: 4
  # of Neurons: 20, Loss: 3.6022408673e-01, R: 5
  # of Neurons: 20, Loss: 3.5893171188e-29, R: 10
  # of Neurons: 30, Loss: 2.9631587752e-29, R: 1
  # of Neurons: 30, Loss: 3.1554436209e-29, R: 2
  # of Neurons: 30, Loss: 1.3016204936e-29, R: 4
  # of Neurons: 30, Loss: 3.6188994027e-29, R: 5
  # of Neurons: 30, Loss: 1.0649622220e-29, R: 10
  # of Neurons: 50, Loss: 8.0266597106e-29, R: 1
  # of Neurons: 50, Loss: 5.1275958839e-30, R: 2
  # of Neurons: 50, Loss: 3.1061398143e-28, R: 4
  # of Neurons: 50, Loss: 1.4002281068e-29, R: 5
  # of Neuron

# Building Mass matrix

In [27]:
from scipy.integrate import quad

def mass_matrix(func,sigma, weights, biases, neurons):
    M = np.zeros([neurons, neurons])
    
    for i in range(0, neurons):
        for j in range(0, neurons):
            M[i, j] = quad(func, 0, 1, args=(weights, biases,i,j,sigma), limit=1000)[0]

    return M

In [28]:
neurons = 10
R = 10
weights = np.random.uniform(-R,R,neurons)
biases = np.random.uniform(-R,R,neurons)
sigma = tanh_derivative
M = mass_matrix(double_neural_net,sigma, weights, biases, neurons)

In [29]:
M

array([[8.14083371e-04, 2.63341331e-05, 2.92212164e-03, 2.00910030e-09,
        4.47134804e-08, 6.01008130e-03, 3.49943224e-06, 7.82125665e-03,
        6.05666037e-05, 1.81567508e-09],
       [2.63341331e-05, 1.13340412e-01, 5.20604622e-03, 9.19080077e-07,
        1.24227430e-08, 1.70907484e-04, 3.46374235e-02, 5.19942089e-04,
        1.29322006e-01, 1.37987206e-06],
       [2.92212164e-03, 5.20604622e-03, 2.31868604e-01, 2.06402451e-07,
        2.94663154e-07, 2.36118249e-02, 5.00618247e-04, 4.05127345e-02,
        1.36342837e-02, 2.25756872e-07],
       [2.00910030e-09, 9.19080077e-07, 2.06402451e-07, 7.88375438e-12,
        3.00637122e-13, 1.54471169e-08, 2.76961657e-07, 2.84146275e-08,
        1.10562806e-06, 1.16656303e-11],
       [4.47134804e-08, 1.24227430e-08, 2.94663154e-07, 3.00637122e-13,
        2.58632168e-12, 3.34534977e-07, 2.24281836e-09, 4.46293014e-07,
        2.36425378e-08, 3.32133672e-13],
       [6.01008130e-03, 1.70907484e-04, 2.36118249e-02, 1.54471169e-08,
   

In [30]:
det_m = np.linalg.det(M)
det_m

1.7141312609510505e-63

In [31]:
b = b_matrix(b_neural_net,sigma,weights, biases,f,neurons)
b

array([[3.18680768e-02],
       [3.98095290e-01],
       [8.21019731e-01],
       [4.17406086e-06],
       [2.31103579e-06],
       [2.47437674e-01],
       [1.00914268e-01],
       [3.61869757e-01],
       [5.66074568e-01],
       [5.77661957e-06]])

In [32]:
a = np.linalg.solve(M,b)
a

array([[ 4.16463341e+02],
       [ 4.23484024e+01],
       [-4.62908029e-01],
       [ 4.94178548e+07],
       [-9.30213113e+06],
       [-1.04493794e+02],
       [ 1.83037329e+02],
       [ 9.10911228e+01],
       [ 1.42404757e+01],
       [-4.19783172e+07]])

In [33]:
collocation = 10
u = u_nn(x,weights,biases,sin_function,collocation,neurons,a)

/tmp/ipykernel_5294/1981966750.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  u[j] += single_neural_net(x[j], weights, biases, i, sigma)*a[i]


In [34]:
u

array([13505607.12224047, 13703751.40247765, 13892622.30898013,
       14072439.92721346, 14243430.12223593, 14405824.05157028,
       14559857.67527023, 14705771.26383874, 14843808.90465736,
       14974218.00758182])

In [35]:
x = np.linspace(0,1,collocation)
loss = l2_loss(f(x), u)
print("L2 Loss:", loss)

L2 Loss: 2044450187260788.2
